# Gradients: A four-term parameter-shift rule (200 points)

In the last problem, you learned about the two-term parameter-shift rule for the Pauli rotation gates. The parameter-shift rule isn't limited to the Pauli rotation gates, and in this problem you'll adventure into more complicated parameter shift rules!

Arguably the next simplest parameter-shift rule compared to the Pauli rotation gates is that of the *controlled* Pauli rotation gates: $CRX$, $CRY$, and $CRZ$. Let's highlight the key differences between the parameter-shift rule for the controlled Pauli rotations and the regular Pauli rotations.

- Four terms: the parameter-shift rule for the controlled Pauli rotations contains four terms, not two!
- Shifts: for the parameter-shift rule for the Pauli rotations, we could use any real-valued shift $s$ for it to work. Here, it's not so simple! The coefficients of the shift rule and the shifts themselves depend on each other!
The parameter-shift rule looks something like this:

$\frac{\partial }{\partial \theta _{i}} f(\vec{\theta} )=c_{1}(f(\theta _{i}+s _{1})-f(\theta _{i}-s _{1}))-c_{2}(f(\theta _{i}+s _{2})-f(\theta _{i}-s _{2}))$

where the shifts $s_{i}$ and coefficients $c_{i}$ depend on each other. You'll have to do some digging to figure out how to calculate them... but there isn't a unique solution!

### Challenge code

In the code below, you are given a few functions:

- `circuit`: This describes the function $f(\vec{\theta} )$.
- `shifts_and_coeffs`: This function calculates $s_{1}$, $s_{2}$, $c_{1}$, and $c_{2}$. No fancy calculations are needed here; you can simply state, for example, $s_{1}=1.23$ and $s_{2}=4.56$ and return those values. **You must complete this function.**
- `my_parameter_shift_grad`: This is where you will implement calculating $\frac{\partial }{\partial \theta _{i}} f(\vec{\theta} )$ for every $\theta _{i}\in \vec{\theta }$. It will return the gradient, which is a vector containing every partial derivative. **You must complete this function.**

#### Input
As input to this problem, you are given `params (list(float))`, which correspond to the differentiable circuit parameters $\vec{\theta}$.
#### Output

This code will output the `gradient (list(float)) of the `circuit`.

If your solution matches the correct one within the given tolerance specified in `check` (in this case it's a `1e-4` relative error tolerance), the output will be `"Correct!"`. Otherwise, you will receive a `"Wrong answer"` prompt.

Good luck!

In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def circuit(params):
    """The quantum circuit that you will differentiate!

    Args:
        params (list(float)): The parameters for gates in the circuit
    
    Returns:
        (numpy.array): An expectation value. 
    """
    qml.broadcast(qml.Hadamard, wires=range(3), pattern="single")
    qml.CRX(params[0], [1, 2])
    qml.CRY(params[1], [0, 1])
    qml.CRZ(params[2], [2, 0])
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1) + qml.PauliX(2))

In [3]:
def shifts_and_coeffs():
    """A function that defines the shift amounts and coefficients needed for
    defining a parameter-shift rule for CRX, CRY, and CRZ gates.

    Returns:
        shifts (list(float)): A list of shift amounts. Order them however you want!
        coeffs (list(float)): A list of coefficients. Order them however you want!
    """
    # Put your code here and make sure to return what is needed #
    s1 = np.pi/2
    s2 = 3*np.pi/2
    c1 = (math.sqrt(2)+1)/4/math.sqrt(2)
    c2 = (math.sqrt(2)-1)/4/math.sqrt(2)
    return [s1,s2],[c1,c2]

def my_parameter_shift_grad(params):
    """Your homemade parameter-shift rule function!
    NOTE: you cannot use qml.grad within this function

    Args:
        params (list(float)): The parameters for gates in the circuit
    
    Returns:
        gradient (numpy.array): The gradient of the circuit with respect to the given parameters.
    """
    gradient = np.zeros_like(params)

    shifts, coeffs = shifts_and_coeffs()

    for i in range(len(params)):
        # Put your code here
        shifted_params1_add = np.copy(params)
        shifted_params1_add[i] += shifts[0]
        shifted_params1_minus = np.copy(params)
        shifted_params1_minus[i] -= shifts[0]
        
        shifted_params2_add = np.copy(params)
        shifted_params2_add[i] += shifts[1]
        shifted_params2_minus = np.copy(params)
        shifted_params2_minus[i] -= shifts[1]
        
        gradient[i] = (coeffs[0]*(circuit(shifted_params1_add)-circuit(shifted_params1_minus))-
                       coeffs[1]*(circuit(shifted_params2_add)-circuit(shifted_params2_minus)))
    
    return np.round_(gradient, decimals=5).tolist()

In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    params = json.loads(test_case_input)
    gradient = my_parameter_shift_grad(params)
    return str(gradient)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your gradient isn't quite right!"

In [5]:
test_cases = [['[1.23, 0.6, 4.56]', '[0.08144, -0.33706, -0.37944]']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[1.23, 0.6, 4.56]'...
Correct!
